In [17]:
import arcpy
from arcpy.sa import Slope, Aspect, Con, ExtractByMask
import os
import datetime
import csv
import numpy as np
import matplotlib
matplotlib.use("Agg")  # Safe backend (no GUI)
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap, BoundaryNorm
from matplotlib.patches import Patch

In [18]:
# Let ArcGIS overwrite outputs
arcpy.env.overwriteOutput = True

# Check Spatial Analyst
if arcpy.CheckExtension("Spatial") == "Available":
    arcpy.CheckOutExtension("Spatial")
else:
    raise RuntimeError("Spatial Analyst not available")


In [19]:
# helper printing function
def msg(text):
    arcpy.AddMessage(text)
    print(text)


In [20]:
# ensure that the geodatabase exists
def ensure_gdb_exists(gdb_path):
    folder, gdb_name = os.path.split(gdb_path)
    if not os.path.exists(folder):
        raise ValueError(f"Parent folder does not exist: {folder}")

    if not arcpy.Exists(gdb_path):
        msg(f"Creating geodatabase: {gdb_path}")
        arcpy.management.CreateFileGDB(folder, gdb_name)
    else:
        msg(f"Using existing geodatabase: {gdb_path}")

In [21]:
#  user parameters for the function
polygon_fc = r"C:\GEOG3700\finalproject\my_plot.gdb\yard_poly"
dem_raster = r"C:\GEOG3700\finalproject\my_dem.tif"

output_gdb = r"C:\GEOG3700\finalproject\sunshade_outputs.gdb"
output_folder = r"C:\GEOG3700\finalproject"

project_name = "myplot"

timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")

# Define aspect-to-sunlight category mapping
sun_category_defs = {
    1: "Full Sun (South-facing; 135–225°)",
    2: "Part Sun (East & West; 45–135° & 225–315°)",
    3: "Mostly Shade (North-facing; 315–360° & 0–45°)"
}

In [22]:
# MAIN FUNCTION BELOW

# Get parameters from the geoprocessing UI
# polygon_fc = arcpy.GetParameterAsText(0)
# dem_raster = arcpy.GetParameterAsText(1)
# output_gdb = arcpy.GetParameterAsText(2)
# output_folder = arcpy.GetParameterAsText(3)
# project_name = arcpy.GetParameterAsText(4)

# Ensure output locations exist and are valid
ensure_gdb_exists(output_gdb)

# Checking if polygon exiszts
if not arcpy.Exists(polygon_fc):
    arcpy.AddError(f"Input polygon not found: {polygon_fc}")
    raise FileNotFoundError(polygon_fc)

# check if the DEM exists
if not arcpy.Exists(dem_raster):
    arcpy.AddError(f"Input DEM not found: {dem_raster}")
    raise FileNotFoundError(dem_raster)

Using existing geodatabase: C:\GEOG3700\finalproject\sunshade_outputs.gdb


In [23]:
# STEP 1 CLIP DEM TO POLYGON   
clipped_dem_name = f"{project_name}_dem_clip_{timestamp}"
clipped_dem_path = os.path.join(output_gdb, clipped_dem_name)

In [24]:
msg("Clipping DEM to polygon ...")
try:
    clipped = ExtractByMask(dem_raster, polygon_fc)
    clipped.save(clipped_dem_path)
    msg(f"  -> Clipped DEM saved to: {clipped_dem_path}")
except Exception as e:
    print(e)
    clipped.save(clipped_dem_path)

Clipping DEM to polygon ...
  -> Clipped DEM saved to: C:\GEOG3700\finalproject\sunshade_outputs.gdb\myplot_dem_clip_20251209_143821


In [25]:
# STEP 2 COMPUTE SLOPE AND ASPECT

slope_name = f"{project_name}_slope_{timestamp}"
aspect_name = f"{project_name}_aspect_{timestamp}"

slope_path = os.path.join(output_gdb, slope_name)
aspect_path = os.path.join(output_gdb, aspect_name)

In [26]:
# computer slope
msg("Computing slope (degrees) ...")
slope_raster = Slope(clipped_dem_path, output_measurement="DEGREE")
slope_raster.save(slope_path)
msg(f"  -> Slope raster saved to: {slope_path}")

Computing slope (degrees) ...
  -> Slope raster saved to: C:\GEOG3700\finalproject\sunshade_outputs.gdb\myplot_slope_20251209_143821


In [27]:
# compute aspect
msg("Computing aspect (degrees) ...")
aspect_raster = Aspect(clipped_dem_path)
aspect_raster.save(aspect_path)
msg(f"  -> Aspect raster saved to: {aspect_path}")

Computing aspect (degrees) ...
  -> Aspect raster saved to: C:\GEOG3700\finalproject\sunshade_outputs.gdb\myplot_aspect_20251209_143821


In [28]:
# STEP 3 CLASSIFY SUNLIGHT
sun_name = f"{project_name}_sun_exposure_{timestamp}"
sun_path = os.path.join(output_gdb, sun_name)

msg("Classifying sunlight exposure from aspect ...")

aspect = arcpy.Raster(aspect_path)

Classifying sunlight exposure from aspect ...


In [29]:
# ArcGIS Aspect:
#   -1 = flat
#   0–360 = compass direction of maximum slope (0 = north, 90 = east, etc.)

full_sun = ((aspect >= 135) & (aspect < 225))    # south-facing
part_sun = (((aspect >= 45) & (aspect < 135)) |  # east
            ((aspect >= 225) & (aspect < 315)))  # west
shade = ((aspect >= 315) | (aspect < 45))        # north

sun_raster = Con(full_sun, 1, Con(part_sun, 2, Con(shade, 3)))

sun_raster.save(sun_path)
msg(f"  -> Sunlight exposure raster saved to: {sun_path}")

  -> Sunlight exposure raster saved to: C:\GEOG3700\finalproject\sunshade_outputs.gdb\myplot_sun_exposure_20251209_143821


In [30]:
# STEP 4 EXPORT THE PNG
msg("Exporting sunlight exposure PNG ...")

if not os.path.exists(output_folder):
    msg(f"Creating output folder for CSV and PNG: {output_folder}")
    os.makedirs(output_folder)

sun_raster = arcpy.Raster(sun_path)
arr = arcpy.RasterToNumPyArray(sun_raster, nodata_to_value=-9999)

# Mask NoData cells
mask = arr == -9999
arr_masked = np.ma.masked_where(mask, arr)

fig, ax = plt.subplots(figsize=(8, 6))

# Define category → color mapping
sun_colors = {
    1: "#FDB813",  # full sun (yellow)
    2: "#8ECFC9",  # partial sun (teal)
    3: "#4B6F44",  # shade (dark green)
}

sun_labels = {
    1: "Full Sun (S-facing)",
    2: "Partial Sun (E/W)",
    3: "Shade (N-facing)",
}

# Create ordered color list (index = value-1)
cmap = ListedColormap([sun_colors[1], sun_colors[2], sun_colors[3]])

# Define boundaries so values map correctly
norm = BoundaryNorm([0.5, 1.5, 2.5, 3.5], cmap.N)

im = ax.imshow(arr_masked, cmap=cmap, norm=norm)

# Build a legend from unique category values
unique_vals = np.unique(arr[~mask])
from matplotlib.patches import Patch
patches = []

for v in unique_vals:
    idx = int(v) % cmap.N
    label = sun_category_defs.get(int(v), f"Category {v}")
    patches.append(Patch(color=cmap(idx), label=label))

ax.legend(handles=patches, loc="lower right", title="Sunlight Categories")
ax.set_title(f"Sunlight Exposure: {project_name}")
ax.axis("off")

timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
png_name = f"{project_name}_sunlight_map_{timestamp}.png"
png_path = os.path.join(output_folder, png_name)

plt.tight_layout()
plt.savefig(png_path, dpi=400)
plt.close(fig)

msg(f"  -> Sunlight exposure PNG saved to: {png_path}")

Exporting sunlight exposure PNG ...
  -> Sunlight exposure PNG saved to: C:\GEOG3700\finalproject\myplot_sunlight_map_20251209_143848.png
